In [1]:
import os
import pandas as pd
import numpy as np

## Loading and Preprocessing Data

In [2]:
ls

CFRC_Amazon-Copy1.ipynb  ExploringData.ipynb      movies_history.pkl
CFRC_Amazon.ipynb        __pycache__/             movies_history_1.pkl
CFRC_Books-2.ipynb       bookDataset/             movies_history_2.pkl
CFRC_Books.ipynb         books_logs/              movies_logs/
CFRC_BooksF.ipynb        makeRec.py               prepdata.py
CFRC_Movies.ipynb        ml-latest-small/         ratings_Electronics.csv
CFRC_MoviesF.ipynb       model01.h5               recmodel.py


In [3]:
from prepdata import load

In [4]:
import os
#book dir holds both ratings.csv and books.csv
books_dir= os.path.abspath(os.path.abspath("bookDataset"))
amazon_file = os.path.abspath(os.path.abspath("ratings_Electronics.csv"))
#movie dir holds both  ratings.csv and movies.csv
movies_dir = os.path.abspath(os.path.abspath("ml-latest-small"))

In [6]:
df = load("amazon", amazon_file)

In [7]:
df

,userId,itemId,rating
0,A2CX7LUOHB2NDG,0321732944,5.0
1,A2NWSAGRHCP8N5,0439886341,1.0
2,A2WNBOD3WNDNKT,0439886341,3.0
3,A1GI0U4ZRJA8WN,0439886341,1.0
4,A1QGNMC6O1VW39,0511189877,5.0
...,...,...,...
7824476,A2YZI3C9MOHC0L,BT008UKTMW,5.0
7824477,A322MDK0M89RHN,BT008UKTMW,5.0
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0


In [8]:
from prepdata import createList, formatData

In [9]:
user = createList(df, pram= 'user')
item = createList(df, pram= 'item')

In [10]:
#error with amazon dataset
num_users, num_items = formatData(df, user,item)

Number of users: 4201696, Number of Books: 476001, Min rating: 1.0, Max rating: 5.0


In [13]:
#will used later to create a function
num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["rating"] = df["rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(df["rating"])
max_rating = max(df["rating"])

print(
    "Number of users: {}, Number of Products: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)


Number of users: 4201696, Number of Products: 476002, Min rating: 1.0, Max rating: 5.0


# Prepare training and validation data

In here, we shuffle the data indices and made a 90/10 split for test/val. 

In [12]:
from prepdata import prepData

In [13]:
x_train, x_val, y_train, y_val = prepData(df)

# Create the model

We embed both users and movies in to 50-dimensional vectors.

The model computes a match score between user and movie embeddings via a dot product, and adds a per-movie and per-user bias. The match score is scaled to the [0, 1] interval via a sigmoid (since our ratings are normalized to this range).

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
EMBEDDING_SIZE = 50


class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_items, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)


Metal device set to: Apple M1 Pro


2022-05-06 11:02:44.021229: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-06 11:02:44.021678: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/citlallylopez/miniforge3/envs/mbp/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Train the model based on the data split

In [22]:
#saving checkoints
from keras.callbacks import ModelCheckpoint

#issue: unable to use due to subclassed model
checkpoint = ModelCheckpoint('amazon_logs/model{epoch:02d}.h5', monitor='loss', verbose=1,
     save_best_only=True, mode='auto', save_freq='epoch', save_weights_only=True)

In [39]:
#RecommenderNet
#from recmodel import RecommenderNet

In [45]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=2,
    verbose=1,
    validation_data=(x_val, y_val),
)
#started arround 5:40pm//interrupted at 8:58pm

Epoch 1/2


/Users/citlallylopez/miniforge3/envs/mbp/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:436: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 210084800 elements. This may consume a large amount of memory.
  warnings.warn(
2022-05-01 19:41:11.589488: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  8998/110032 [=>............................] - ETA: 14:37:19 - loss: 0.6612

KeyboardInterrupt: 

### Load and save history

In [ ]:
import pickle
# Save train  loss history - changes previous history if trained before
pickle.dump(history, open('amazon_history.pkl', 'wb'))

In [ ]:
#if training before, input old history
with open('amazon_history.pkl', 'rb') as f:
    h = pickle.load(f)

In [ ]:
h.history

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()
### Load and save history

import pickle
# Save train  loss history - changes previous history if trained before
pickle.dump(history, open('books_history.pkl', 'wb'))

#if training before, input old history
with open('books_history.pkl', 'rb') as f:
    h = pickle.load(f)

h.history

In [ ]:
#To lookup product use "https://amazon-asin.com/asincheck/?product_id="+ product_id

# Show top 10 prodcuts recommendations to a user

In [ ]:
# Let us get a user and see the top recommendations.
uid = df.user_id.sample(1).iloc[0]
items_by_user = df[df.user_id == uid]
items_not_watched = df[
    ~df["prodcutId"].isin(items_by_user.book_id.values)
]["productId"]
items_not_watched = list(
    set(items_not_watched).intersection(set(item2item_encoded.keys()))
)
items_not_watched = [[item2item_encoded.get(x)] for x in items_not_watched]
user_encoder = user2user_encoded.get(uid)
user_item_array = np.hstack(
    ([[user_encoder]] * len(items_not_watched), items_not_watched)
)
ratings = model.predict(user_item_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_item_ids = [
    item_encoded2item.get(items_not_watched[x][0]) for x in top_ratings_indices
]


In [ ]:
print("Showing recommendations for user: {}".format(uid))
print("====" * 9)
print("Product with high ratings from user")
print("----" * 8)
top_item_user = (
    items_by_user.sort_values(by="rating", ascending=False)
    .head(5)
    .productId.values
)
items_df_rows = df[df["productId"].isin(top_item_user)]
for row in items_df_rows.itertuples():
    print(row.productId, "with rating of ", row.rating)

print("----" * 8)
print("Top 10 product recommendations")
print("----" * 8)
recommended_items = items_df[items_df["productId"].isin(recommended_item_ids)]
for row in recommended_items.itertuples():
    print(row.productId)